In [49]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch
from tqdm import tqdm
from datasets import load_dataset


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_id = 'roberta-base'
model = AutoModelForMaskedLM.from_pretrained(model_id).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)

test_sentences = []  #replace this with your list of sentence

if test_sentences == []:
    test_sentences = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')['text'][:1]
    
encodings = tokenizer('\n\n'.join(test_sentences), return_tensors='pt')

max_length = 512
stride = 512

lls = []
for i in tqdm(range(0, encodings.input_ids.size(1), stride)):
    begin_loc = max(i + stride - max_length, 0)
    end_loc = min(i + stride, encodings.input_ids.size(1))
    trg_len = end_loc - i    # may be different from stride on last loop
    input_ids = encodings.input_ids[:,begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:,:-trg_len] = -100   #invalidate the context tokens

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)
        log_likelihood = outputs[0] * trg_len

    lls.append(log_likelihood)

ppl = torch.exp(torch.stack(lls).sum() / end_loc)
print(f"The average perplexity is: {ppl}")

Reusing dataset wikitext (C:\Users\hoang\.cache\huggingface\datasets\wikitext\wikitext-2-raw-v1\1.0.0\aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20)
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 90.94it/s]

The average perplexity is: 1.0010002851486206


In [45]:
test_sentences

['']